In [377]:
import pandas as pd
import altair as alt
import altair_latimes as lat

In [378]:
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

### Import

Monthly reports timeseries

In [379]:
df = pd.read_csv("../data/raw/uw-usage.csv", parse_dates=["Reporting Month"])

In [380]:
clean_names = pd.read_csv("../data/metadata/urban-water-suppliers-clean-names.csv")

### Clean

Remove junk from column names

In [381]:
df.columns = df.columns.str.replace(" ","_").str.replace("-","_").str.replace(r'[#,@,&,(,)]', '', regex=True).str.lower()

Eliminate double spaces in supplier names

In [382]:
df.supplier_name = df.supplier_name.str.replace("  ", " ")

In [383]:
df[
    df.reporting_month == df.reporting_month.max()
].total_population_served.sum()

35007792

Trim this downn to just the columns we need

### Merge clean names

In [384]:
merge_df = pd.merge(
    df,
    clean_names,
    how="left",
    left_on=["public_water_system_id" , "supplier_name"],
    right_on=["id" , "supplier_name"]
)

### Trim

In [385]:
keeps = [
    'supplier_name', 
    'display_name',
    'public_water_system_id', 
    'reporting_month', 
    'county',
    'hydrologic_region', 
    # 'climate_zone', 
    'total_population_served',
    ### MAYBE PILE
        'county_under_drought_declaration',
        'water_shortage_contingency_stage_invoked',
        'water_shortage_level_indicator',
        'dwr_state_standard_level_corresponding_to_stage',
    ### NEED THIS IF YOU WANT TO RECALCULATE R-GPCD BY DISTRICT
        # 'water_production_units',
        # 'reported_preliminary_total_potable_water_production',
        # 'reported_final_total_potable_water_production',
        # 'preliminary_percent_residential_use', 
        # 'final_percent_residential_use',
    ### THESE COLUMNS ARE OPTIONAL FOR SUPPLIERS TO FILL OUT
        # 'reported_preliminary_commercial_agricultural_water',
        # 'reported_final_commercial_agricultural_water',
        # 'reported_preliminary_commercial,_industrial_and_institutional_water',
        # 'reported_final_commercial_industrial_and_institutional_water',
        # 'reported_recycled_water', 
        # 'reported_non_revenue_water',
    'calculated_total_potable_water_production_gallons_ag_excluded',
    # 'calculated_total_potable_water_production_gallons_2013_ag_excluded',
    # 'calculated_commercial_agricultural_water_gallons',
    # 'calculated_commercial_agricultural_water_gallons_2013',
    'calculated_r_gpcd', 
    # 'qualification'
]

In [386]:
trim_df = merge_df[keeps]

In [387]:
trim_df.dwr_state_standard_level_corresponding_to_stage.unique()

array([ 5., nan,  2.,  3.,  0.,  1.,  4.])

### Backfill missing dates

In [388]:
min_date = trim_df.reporting_month.min()
min_date

Timestamp('2014-06-15 00:00:00')

In [389]:
max_date = trim_df.reporting_month.max()
max_date

Timestamp('2022-04-15 00:00:00')

In [390]:
def backfill(agency_group):
    """
    Backfills empty dates in the provided county group.

    Runs from the earliest date in the group to the latest.

    Filled in dates are given the previous day's case count with an `ffill` technique.

    The expanded group is returned.
    """
    agency_df = agency_group.sort_values(["supplier_name", "reporting_month"]).set_index(
        ["supplier_name", "reporting_month"]
    )

    # Backfill the daterange
    ## Get the full range of values from the extent of dates in the dataframe
    date_range = pd.date_range(
        min_date,
        max_date,
        freq=pd.DateOffset(months=1, day=15),
    )
    ## Get the full range of unique place names
    name_range = agency_df.index.unique(level="supplier_name")
    ## Create a new index that has an entry for every place on every date
    namedate_index = pd.MultiIndex.from_product(
        iterables=[name_range, date_range], names=["supplier_name", "reporting_month"]
    )
    ## Reindex the dataframe using that complete list of places and dates
    backfilled_df = agency_df.reindex(namedate_index)

    # Zero out missing data
    backfilled_df.calculated_r_gpcd.fillna(-999999999, inplace=True)
    backfilled_df.calculated_total_potable_water_production_gallons_ag_excluded.fillna(-999999999, inplace=True)

    # Foward-fill the other remaining columns
    backfilled_df = backfilled_df.groupby("supplier_name").ffill()

    # Reset it
    reset_df = backfilled_df.reset_index()

    # Return it
    return reset_df

In [391]:
backfilled_df = (
    trim_df.groupby("supplier_name").apply(backfill).reset_index(drop=True)
)

In [392]:
backfilled_df[backfilled_df.display_name=="City of Pismo Beach"]

,supplier_name,reporting_month,display_name,public_water_system_id,county,hydrologic_region,total_population_served,county_under_drought_declaration,water_shortage_contingency_stage_invoked,water_shortage_level_indicator,dwr_state_standard_level_corresponding_to_stage,calculated_total_potable_water_production_gallons_ag_excluded,calculated_r_gpcd
26315,Pismo Beach City of,2014-06-15,City of Pismo Beach,CA4010008,San Luis Obispo,Central Coast,7705.0,NaN,3rd,NaN,NaN,6.328678e+07,1.258010e+02
26316,Pismo Beach City of,2014-07-15,City of Pismo Beach,CA4010008,San Luis Obispo,Central Coast,7705.0,NaN,3rd,NaN,NaN,6.685159e+07,1.286460e+02
26317,Pismo Beach City of,2014-08-15,City of Pismo Beach,CA4010008,San Luis Obispo,Central Coast,7705.0,NaN,3rd,NaN,NaN,5.372957e+07,1.035450e+02
26318,Pismo Beach City of,2014-09-15,City of Pismo Beach,CA4010008,San Luis Obispo,Central Coast,7705.0,NaN,3rd,NaN,NaN,5.375890e+07,1.069960e+02
26319,Pismo Beach City of,2014-10-15,City of Pismo Beach,CA4010008,San Luis Obispo,Central Coast,7705.0,NaN,3rd,NaN,NaN,5.493848e+07,1.060550e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26405,Pismo Beach City of,2021-12-15,City of Pismo Beach,CA4010008,San Luis Obispo,Central Coast,8233.0,Yes,2nd of 4,No,NaN,3.880885e+07,6.988830e+01
26406,Pismo Beach City of,2022-01-15,City of Pismo Beach,CA4010008,San Luis Obispo,Central Coast,8233.0,Yes,3rd of 4,Yes,NaN,3.497359e+07,6.284080e+01
26407,Pismo Beach City of,2022-02-15,City of Pismo Beach,CA4010008,San Luis Obispo,Central Coast,8233.0,Yes,3rd of 4,Yes,NaN,-1.000000e+09,-1.000000e+09
26408,Pismo Beach City of,2022-03-15,City of Pismo Beach,CA4010008,San Luis Obispo,Central Coast,8233.0,Yes,3rd of 4,Yes,NaN,-1.000000e+09,-1.000000e+09


### Calculate

Calculate population-weighted r-gpcd for hydrologic regions

In [393]:
def regional_calcs(df, gals, rgpcd, pop):
    val = df[rgpcd]
    wt = df[pop]
    wt_avg = (val * wt).sum() / wt.sum()
    #return (val * wt).sum() / wt.sum()
    total_gals = df[gals].sum()
    total_pop = df[pop].sum()
    return pd.Series([total_pop, total_gals, wt_avg], index=['total_pop', 'total_gallons', 'pop_weighted_rgpcd'])

In [394]:
region_df = backfilled_df.groupby(
    ['reporting_month','hydrologic_region']
).apply(
    regional_calcs,
    "calculated_total_potable_water_production_gallons_ag_excluded",
    'calculated_r_gpcd', 
    'total_population_served'
).reset_index()

Now do it for the entire state

In [395]:
statewide_df = backfilled_df.groupby(
    ['reporting_month']
).apply(
    regional_calcs,
    "calculated_total_potable_water_production_gallons_ag_excluded",
    'calculated_r_gpcd', 
    'total_population_served'
).reset_index()

In [396]:
# in case we need to recalculate r-gpcd, use this dict for days per month
# days_per_month = {
#     "1": 31,
#     "2": 28,
#     "3": 31,
#     "4": 30,
#     "5": 31,
#     "6": 30,
#     "7": 31,
#     "8": 31,
#     "9": 30,
#     "10": 31,
#     "11": 30,
#     "12": 31
# }

### Merge regional r-gpcd values to district df

In [397]:
merge_df = pd.merge(
    backfilled_df, 
    region_df[["hydrologic_region","reporting_month","pop_weighted_rgpcd"]], 
    how="left", 
    on=["hydrologic_region","reporting_month"]
)

### Rename r-gpcd columns for clarity

In [398]:
rename_df = merge_df.rename(columns={
    "calculated_r_gpcd": "r_gpcd",
    "pop_weighted_rgpcd": "regional_r_gpcd"
})

### Chart

In [399]:
melt = pd.melt(
    rename_df, 
    id_vars=["display_name","hydrologic_region","reporting_month"], 
    value_vars=["r_gpcd","regional_r_gpcd"]
)

In [400]:
agency_name = "City of Pismo Beach"

base = alt.Chart(
    rename_df[
        (rename_df.display_name == agency_name)
    ].tail(12)
).encode(
    x=alt.X("yearmonth(reporting_month):O"),
    tooltip=["reporting_month"]
)

bar = base.mark_bar(color="#83c6e0").encode(
    y=alt.Y("r_gpcd", stack=None)
)

avg_line = base.mark_line(interpolate='step', color='#1281aa').encode(
    y=alt.Y("regional_r_gpcd")
)

goal_line = alt.Chart(pd.DataFrame({'y': [80]})).mark_rule(color="#b75a36",strokeDash=[10,11]).encode(y='y')

(bar + avg_line + goal_line).properties(title=f"{agency_name} residential water usage compared to regional average", width=600)

alt.LayerChart(...)

### Sort data

In [401]:
sort_district_df = merge_df.sort_values(["reporting_month","supplier_name"])

In [402]:
sort_region_df = region_df.sort_values(["reporting_month","hydrologic_region"])

In [403]:
sort_state_df = statewide_df.sort_values(["reporting_month"])

### Filter dataframe to be a bit more manageable

In [404]:
filtered_district_df = sort_district_df[sort_district_df.reporting_month >= min_date]

### Export

In [405]:
filtered_district_df.to_csv("../data/processed/district-level-residential-use.csv", index=False)

In [406]:
filtered_district_df[
    filtered_district_df.reporting_month == filtered_district_df.reporting_month.max()
].to_csv("../data/processed/latest-district-level-residential-use.csv", index=False)

In [407]:
sort_region_df.to_csv("../data/processed/regional-residential-usage.csv", index=False)

In [408]:
sort_region_df[
    sort_region_df.reporting_month == sort_district_df.reporting_month.max()
].to_csv("../data/processed/latest-regional-residential-use.csv", index=False)

In [409]:
sort_state_df.to_csv("../data/processed/statewide-residential-usage.csv", index=False)

In [410]:
sort_state_df[
    sort_state_df.reporting_month == sort_district_df.reporting_month.max()
].to_csv("../data/processed/latest-statewide-level-residential-use.csv", index=False)